In [1]:
import sys
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
import os

In [2]:
# Оператор Робертса
roberts_cross_v = np.array( [[ 0, 0, 0 ],
                             [ 0, 1, 0 ],
                             [ 0, 0,-1 ]] )

roberts_cross_h = np.array( [[ 0, 0, 0 ],
                             [ 0, 0, 1 ],
                             [ 0,-1, 0 ]] )

def roberts_cross(image_data) :
    vertical = ndimage.convolve(image_data, roberts_cross_v )
    horizontal = ndimage.convolve(image_data, roberts_cross_h )
    return np.sqrt( np.square(horizontal) + np.square(vertical))

In [3]:
def ShowImage(img, name):
    fig, axes = plt.subplots(1)
    axes.imshow(img, cmap="gray")
    axes.set_title(name)
    axes.axis("off")    
    fig.set_figwidth(10)    
    fig.set_figheight(10)  
    plt.show()

def PrintHist(imageArray):
    x = [i for i in range(0,256)]
    y = np.zeros(256, dtype = int)
    y1 = np.zeros(256, dtype = int)
    y2 = np.zeros(256, dtype = int)
    y3 = np.zeros(256, dtype = int)
    if len(imageArray.shape)>2:
      for i in imageArray[:,:,0].flat:
        y[i] += 1
      for i in imageArray[:,:,1].flat:
        y1[i] += 1
      for i in imageArray[:,:,2].flat:
        y2[i] += 1
      for i in imageArray.flat:
        y3[i] += 1
    else:
      for i in imageArray.flat:
        y3[i] += 1
    return IsLight(y3, imageArray)


def IsLight(lightArray, imgArray):
  lightArray = lightArray / imgArray.size
  treshHold = (np.max(lightArray) * 10)/100
  newLightArray = lightArray > treshHold
  newLightArraySum = newLightArray.sum()
  if newLightArraySum > 7:
    return "5. Тоновое"
  elif newLightArraySum <= 2 and newLightArraySum != 0:
    return "5. Штриховое"
  else:
    return "5. Многоградационное"


def IsGreyScale(img):
  imgRGB = img.convert('RGB')
  w,h = imgRGB.size
  for i in range(w):
    for j in range(h):
      r,g,b = imgRGB.getpixel((i,j))
      if r != g != b: return False
  return True


In [4]:
for root, dirs, files in os.walk("/content/pr3"):  
    for filename in files:
      imgFullPath = f"{root}/{filename}"
      image = Image.open(imgFullPath)

      imgArr = np.asarray(image)
      image_data = np.asarray(image.convert("L"), dtype="int32")
      middle = int(np.median(image_data)) #Средняя яркость пикселей
      lightness = ""
      light_level = 0
      if(middle in range(130,255)):
          lightness = "светлое"
          light_level = 0.78
      elif(middle in range(100,130)):
          lightness = "среднее"
          light_level = 0.46
      elif(middle in range(0,100)):
          lightness = "тёмное"
          light_level = 0.11

      border_mask = image_data > middle*light_level
      image_data[border_mask] = 255
      image_data[np.invert(border_mask)] = 0

      roberts_image_data = roberts_cross(image_data)
      tones_pixels_count = np.count_nonzero(roberts_image_data < 128)
      image_pixels_count = image_data.shape[0] * image_data.shape[1]
      tones_percent = tones_pixels_count / image_pixels_count
      
      print(imgFullPath)
      color = False
      print(f"1. Яркость: {lightness}")
      print(f"2. Площадь, занятая равномерными тонами: {tones_percent}")

      if tones_percent < 0.94:
          print(f"3. Изображение с большим количеством сюжетных деталей")
      elif tones_percent < 0.96:
          print(f"3. Среднее содержание контуров")
      else:
          print(f"3. Преобладают равномерные тона")

      if IsGreyScale(image) == True:
          print("4. Черно-белое")
      else:
          print("4. Цветное")
          color=True

      gr = PrintHist(imgArr)
      print(gr)
      
      if(color == True and gr == "5. Многоградационное"):
          print("Рекомендуется цветоделение")
      elif(gr == "5. Тоновое" or gr == "5. Многоградационное"):
          print("Рекомендуется растрирование")
      elif(color == True):
          print("Рекомендуется цветоделение")
      print()

/content/pr3/07.jpg
1. Яркость: тёмное
2. Площадь, занятая равномерными тонами: 0.806850921064119
3. Изображение с большим количеством сюжетных деталей
4. Цветное
5. Тоновое
Рекомендуется растрирование

/content/pr3/02.jpg
1. Яркость: тёмное
2. Площадь, занятая равномерными тонами: 0.9743784679548074
3. Преобладают равномерные тона
4. Черно-белое
5. Штриховое

/content/pr3/03.jpg
1. Яркость: светлое
2. Площадь, занятая равномерными тонами: 0.9847811377663265
3. Преобладают равномерные тона
4. Цветное
5. Тоновое
Рекомендуется растрирование

/content/pr3/10.jpg
1. Яркость: светлое
2. Площадь, занятая равномерными тонами: 0.9877532991431609
3. Преобладают равномерные тона
4. Цветное
5. Многоградационное
Рекомендуется цветоделение

/content/pr3/01.jpg
1. Яркость: светлое
2. Площадь, занятая равномерными тонами: 0.8824245053833936
3. Изображение с большим количеством сюжетных деталей
4. Черно-белое
5. Штриховое

/content/pr3/09.jpg
1. Яркость: светлое
2. Площадь, занятая равномерными тонами